In [1]:
import acts

In [2]:
import sys

log = open("output_detector_mg.log", "w")
sys.stdout = sys.stderr = log

In [3]:
!python fatras_mg_pythia.py

In [4]:
print("Done")

In [1]:
import uproot

# Load the ROOT file
file = uproot.open("mg_pythia_sim_output/root/background/run_01vertices.root")

# Print all keys (TTree names)
print("Keys:", file.keys())

Keys: ['vertices;1']


In [6]:
tree = file["vertices"]  # update if name is different
# Print all available branches
print("Branches:", tree.keys())

# Load branches
vx = tree["vx"].array()
vy = tree["vy"].array()
vz = tree["vz"].array()

# Print the first 10 vertices across events
counter = 0
for event_vx, event_vy, event_vz in zip(vx, vy, vz):
    for x, y, z in zip(event_vx, event_vy, event_vz):
        print(f"Vertex {counter}: ({x:.2f}, {y:.2f}, {z:.2f})")
        counter += 1
        if counter >= 10:
            break
    if counter >= 10:
        break



# # Load vertex coordinates
# vx = tree["vx"].array()
# vy = tree["vy"].array()
# vz = tree["vz"].array()

# # Convert awkward arrays to plain NumPy arrays
# vx_np = vx.to_numpy()
# vy_np = vy.to_numpy()
# vz_np = vz.to_numpy()

# # Print first 10 vertex positions
# for i in range(10):
#     print(f"Vertex {i}: ({vx[i]:.2f}, {vy[i]:.2f}, {vz[i]:.2f})")

Branches: ['event_id', 'vertex_id', 'process', 'vx', 'vy', 'vz', 'vt', 'outgoing_particles', 'vertex_primary', 'vertex_secondary', 'generation']
Vertex 0: (0.00, 0.00, 0.00)
Vertex 1: (-0.00, -0.00, 0.00)
Vertex 2: (-0.00, 0.00, 0.00)
Vertex 3: (-0.00, 0.00, 0.00)
Vertex 4: (-0.00, 0.00, 0.00)
Vertex 5: (-1.73, 0.46, 9.73)
Vertex 6: (-1.75, 0.47, 9.82)
Vertex 7: (-1.75, 0.47, 9.82)
Vertex 8: (-21.15, -24.76, 30.75)
Vertex 9: (-0.00, 0.00, -0.00)


In [1]:
import uproot
import awkward as ak
import numpy as np

# Input/output
input_file = "mg_pythia_sim_output/root/background/background_vertices.root"
output_file = "shifted_vertices.root"
tree_name = "vertices"

# Load data
with uproot.open(input_file) as file:
    tree = file[tree_name]
    data = tree.arrays()

# Shift logic
vx = data["vx"]
vy = data["vy"]
vz = data["vz"]

angles = np.arctan2(vy, vx)
r_shift = 50.0
vx_shifted = r_shift * np.cos(angles)
vy_shifted = r_shift * np.sin(angles)

mask_zero = (vx == 0) & (vy == 0)
vx_shifted = ak.where(mask_zero, r_shift, vx_shifted)
vy_shifted = ak.where(mask_zero, 0.0, vy_shifted)

# Write only compatible branches
with uproot.recreate(output_file) as fout:
    fout[tree_name] = {
        "event_id": data["event_id"],
        "vertex_id": data["vertex_id"],
        "process": data["process"],
        "vx": vx_shifted,
        "vy": vy_shifted,
        "vz": vz,
        "vt": data["vt"],
        "vertex_primary": data["vertex_primary"],
        "vertex_secondary": data["vertex_secondary"],
        "generation": data["generation"],
        # "outgoing_particles": data["outgoing_particles"],  # excluded: too deeply nested
    }

print(f"Successfully wrote {output_file}")

Successfully wrote shifted_vertices.root
